In [1]:
# Import python packages
import numpy as np
import pandas as pd
from fuzzywuzzy import fuzz
from fuzzywuzzy import process as fwprocess
import rpy2
import rpy2.robjects as ro
from rpy2.robjects.packages import importr
from rpy2.robjects import pandas2ri

In [2]:
# Can uncomment to install R packages as needed
#utils = importr('utils')
#utils.install_packages('nflreadr')
#utils.install_packages('ffscrapr')

In [3]:
# Import R packages
nflreadr = importr('nflreadr')
ffscrapr = importr('ffscrapr')

In [4]:
# Config for the calculation
mfl_id = 60206  #MFL id for league
seasons = [2024,2023,2022]  #Seasons to collect data for
max_week = 17  #Maximum week number to include in calculation
num_off = 7  #Number of offensive players to include in calculation for each game
num_def = 12  #Number of defensive  players to include in calculation for each game
save_label = 'Analytics_Dynasty_League_2025'  #Where to save results

In [5]:
# A mapping from PFR team names to MFL team names

###
team_dic = {
    'ARI' : 'ARI',
    'ATL' : 'ATL',
    'BAL' : 'BAL',
    'BUF' : 'BUF',
    'CAR' : 'CAR',
    'CHI' : 'CHI',
    'CIN' : 'CIN',
    'CLE' : 'CLE',
    'DAL' : 'DAL',
    'DEN' : 'DEN',
    'DET' : 'DET',
    'GB' : 'GBP',
    'HOU' : 'HOU',
    'IND' : 'IND',
    'JAX' : 'JAC',
    'KC' : 'KCC',
    'LA' : 'LAR',
    'LAC' : 'LAC',
    'LV' : 'LVR',
    'MIA' : 'MIA',
    'MIN' : 'MIN',
    'NE' : 'NEP',
    'NO' : 'NOS',
    'NYG' : 'NYG',
    'NYJ' : 'NYJ',
    'PHI' : 'PHI',
    'PIT' : 'PIT',
    'SEA' : 'SEA',
    'SF' : 'SFO',
    'TB' : 'TBB',
    'TEN' : 'TEN',
    'WAS' : 'WAS',
}

In [6]:
# Mappings for positions, both to consider which positions to drop from the analysis and to cleanup PFR positions a bit

###
snap_side_map = {
    'C' : 'DROP',
    'CB' : 'DEF',
    'DB' : 'DEF',
    'DE' : 'DEF',
    'DT' : 'DEF',
    'FB' : 'DROP',  #Change to OFF if want to include FBs
    'FS' : 'DEF',
    'G' : 'DROP',
    'K' : 'DROP',
    'LB' : 'DEF',
    'LS' : 'DROP',
    'NT' : 'DEF',
    'P' : 'DROP',
    'QB' : 'OFF',
    'RB' : 'OFF',
    'SS' : 'DEF',
    'T' : 'DROP',
    'TE' : 'OFF',
    'WR' : 'OFF',
}

snap_position_map = {
    'CB' : 'CB',
    'DB' : 'S',
    'DE' : 'DE',
    'DT' : 'DT',
    'FB' : 'RB',
    'FS' : 'S',
    'LB' : 'LB',
    'NT' : 'DT',
    'QB' : 'QB',
    'RB' : 'RB',
    'SS' : 'S',
    'TE' : 'TE',
    'WR' : 'WR',
}

###
pos_side_map = {
    'CB' : 'DEF',
    'Coach' : 'DROP',
    'DE' : 'DEF',
    'DT' : 'DEF',
    'Def' : 'DROP',
    'LB' : 'DEF',
    'Off' : 'DROP',
    'PK' : 'DROP',
    'PN' : 'DROP',
    'QB' : 'OFF',
    'RB' : 'OFF',
    'S' : 'DEF',
    'ST' : 'DROP',
    'TE' : 'OFF',
    'TMDB' : 'DROP',
    'TMDL' : 'DROP',
    'TMLB' : 'DROP',
    'TMPK' : 'DROP',
    'TMPN' : 'DROP',
    'TMQB' : 'DROP',
    'TMRB' : 'DROP',
    'TMTE' : 'DROP',
    'TMWR' : 'DROP',
    'WR' : 'OFF',
    'XX' : 'DROP'}

In [7]:
# Scrape snap counts from PFR and convert to pandas df
snap_df_r = nflreadr.load_snap_counts(ro.IntVector(seasons))

with (ro.default_converter + pandas2ri.converter).context():
  snap_df = ro.conversion.get_conversion().rpy2py(snap_df_r)

# Cleanup some of the data
snap_df['team'] = snap_df['team'].apply(lambda x: team_dic[x])
snap_df['side'] = snap_df['position'].apply(lambda x: snap_side_map[x])

snap_df = snap_df[ snap_df['side'] != 'DROP' ]

snap_df['position'] = snap_df['position'].apply(lambda x: snap_position_map[x])
snap_df['player_pfr'] = snap_df['player'].apply(lambda x: nflreadr.clean_player_names(x, convert_lastfirst=False)[0])
snap_df['join_string'] = snap_df['player_pfr'] + " " + snap_df['team'] + " " + snap_df['position']

snap_df

Note: nflreadr caches (i.e., stores a saved version) data by default.
If you expect different output try one of the following:
ℹ Restart your R Session or
ℹ Run `nflreadr::.clear_cache()`.
To disable this warning, run `options(nflreadr.verbose = FALSE)` or add it to your .Rprofile
This message is displayed once every 8 hours.


,game_id,pfr_game_id,season,game_type,week,player,pfr_player_id,position,team,opponent,offense_snaps,offense_pct,defense_snaps,defense_pct,st_snaps,st_pct,side,player_pfr,join_string
5,2024_01_ARI_BUF,202409080buf,2024,REG,1,Josh Allen,AlleJo02,QB,BUF,ARI,62.0,1.00,0.0,0.00,0.0,0.00,OFF,Josh Allen,Josh Allen BUF QB
7,2024_01_ARI_BUF,202409080buf,2024,REG,1,Dalton Kincaid,KincDa00,TE,BUF,ARI,52.0,0.84,0.0,0.00,0.0,0.00,OFF,Dalton Kincaid,Dalton Kincaid BUF TE
8,2024_01_ARI_BUF,202409080buf,2024,REG,1,Keon Coleman,ColeKe02,WR,BUF,ARI,45.0,0.73,0.0,0.00,0.0,0.00,OFF,Keon Coleman,Keon Coleman BUF WR
9,2024_01_ARI_BUF,202409080buf,2024,REG,1,James Cook,CookJa01,RB,BUF,ARI,38.0,0.61,0.0,0.00,0.0,0.00,OFF,James Cook,James Cook BUF RB
10,2024_01_ARI_BUF,202409080buf,2024,REG,1,Mack Hollins,HollMa00,WR,BUF,ARI,36.0,0.58,0.0,0.00,10.0,0.37,OFF,Mack Hollins,Mack Hollins BUF WR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
79353,2022_22_KC_PHI,202302120phi,2022,SB,22,Darius Harris,HarrDa07,LB,KCC,PHI,0.0,0.00,2.0,0.03,16.0,0.55,DEF,Darius Harris,Darius Harris KCC LB
79354,2022_22_KC_PHI,202302120phi,2022,SB,22,Deon Bush,BushDe00,S,KCC,PHI,0.0,0.00,1.0,0.01,20.0,0.69,DEF,Deon Bush,Deon Bush KCC S
79355,2022_22_KC_PHI,202302120phi,2022,SB,22,Jack Cochrane,CochJa00,LB,KCC,PHI,0.0,0.00,0.0,0.00,17.0,0.59,DEF,Jack Cochrane,Jack Cochrane KCC LB
79356,2022_22_KC_PHI,202302120phi,2022,SB,22,Nazeeh Johnson,JohnNa01,S,KCC,PHI,0.0,0.00,0.0,0.00,17.0,0.59,DEF,Nazeeh Johnson,Nazeeh Johnson KCC S


In [8]:
# Dict to store data for each season
pos_df = {}

for s in seasons:
    # Scrape player positions from MFL and convert to pandas df
    mfl = ffscrapr.mfl_connect(season=s, league_id=mfl_id)
    
    pos_df_r = ffscrapr.mfl_players(mfl)
    
    with (ro.default_converter + pandas2ri.converter).context():
      pos_df[s] = ro.conversion.get_conversion().rpy2py(pos_df_r)

    # Cleanup some of the data
    pos_df[s]['season'] = s
    pos_df[s]['side'] = pos_df[s]['pos'].apply(lambda x: pos_side_map[x])

    pos_df[s] = pos_df[s][ pos_df[s]['side'] != 'DROP' ]
    
    pos_df[s]['player_mfl'] = pos_df[s]['player_name'].apply(lambda x: nflreadr.clean_player_names(x, convert_lastfirst=True)[0])
    pos_df[s]['join_string'] = pos_df[s]['player_mfl'] + " " + pos_df[s]['team'] + " " + pos_df[s]['pos']

# Merge seasons to single df
pos_df = pd.concat(pos_df, ignore_index=True)

# Sort by id and season for cleaner display
pos_df = pos_df.set_index(['player_id','season']).sort_index(level=[0,1],ascending=[True,True]).reset_index()

# Save a copy of positions
pos_df[['player_id','player_mfl','season','team','side','pos','join_string']].to_csv(f'{save_label}_Positions.csv', index=False)

pos_df

,player_id,season,player_name,pos,age,team,status,draft_team,draft_year,draft_round,...,nfl_id,twitter_username,jersey,college,height,weight,birthdate,side,player_mfl,join_string
0,0800,2022,"Weatherford, Sterling",S,NaN,IND,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,DEF,Sterling Weatherford,Sterling Weatherford IND S
1,0802,2022,"Krull, Lucas",TE,NaN,NOS,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,OFF,Lucas Krull,Lucas Krull NOS TE
2,0803,2022,"Vilain, Luiji",DE,NaN,MIN,R,NA_character_,NA_character_,NA_character_,...,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NA_character_,NaN,DEF,Luiji Vilain,Luiji Vilain MIN DE
3,10143,2022,"Mclendon, Steve",DT,39.1,FA,NA_character_,FA,2009,NA_character_,...,stevemclendon/2507590,NA_character_,96,Troy,75,310,5846.0,DEF,Steve McLendon,Steve McLendon FA DT
4,10220,2022,"Sendejo, Andrew",S,37.4,FA,NA_character_,FA,2010,NA_character_,...,andrewsendejo/2525323,Asendejo,42,Rice,73,210,6460.0,DEF,Andrew Sendejo,Andrew Sendejo FA S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6836,9940,2022,"Woods, Al",DT,37.9,SEA,NA_character_,NOS,2010,4,...,alwoods/496881,NA_character_,96,LSU,76,309,6292.0,DEF,Al Woods,Al Woods SEA DT
6837,9940,2023,"Woods, Al",DT,37.9,NYJ,NA_character_,NOS,2010,4,...,alwoods/496881,NA_character_,96,LSU,76,309,6292.0,DEF,Al Woods,Al Woods NYJ DT
6838,9940,2024,"Woods, Al",DT,37.9,FA,NA_character_,NOS,2010,4,...,alwoods/496881,NA_character_,96,LSU,76,309,6292.0,DEF,Al Woods,Al Woods FA DT
6839,9988,2022,"Brown, Antonio",WR,36.6,FA,NA_character_,PIT,2010,6,...,antoniobrown/2508061,AntonioBrown84,81,Central Michigan,70,185,6765.0,OFF,Antonio Brown,Antonio Brown FA WR


In [9]:
###
def GetClosestJoinString(other_df, this_js):
    # Return closest matching join string using fuzzy matching
    return fwprocess.extract(this_js, other_df['join_string'], scorer=fuzz.ratio)[0][0]

def GetClosestJoinScore(other_df, this_js):
    # Return score of closest match using fuzzy matching
    return fwprocess.extract(this_js, other_df['join_string'], scorer=fuzz.ratio)[0][1]

In [10]:
# Initialize merged df from snap data
df = snap_df[['game_id','season','week','pfr_player_id','player_pfr','team','position','side','join_string','offense_snaps','defense_snaps']].copy()

# Filter out weeks above max week
df = df[ df['week'] <= max_week ]

# Merge in positions using fuzzy string matching, considering each season and side of ball separately
df = df.set_index(['pfr_player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])

for s in seasons:
    for side in ['OFF','DEF']:
        print(s,side)

        # Constrain data to this season and side of ball
        tmp_snap_df = snap_df[ (snap_df['season'] == s) & (snap_df['side'] == side) ]
        tmp_snap_df = tmp_snap_df.set_index(['pfr_player_id','season','week']).sort_index(level=[0,1,2],ascending=[True,True,True])
        tmp_snap_df = tmp_snap_df.groupby(['pfr_player_id','season']).tail(1).droplevel(2)

        tmp_pos_df = pos_df[ (pos_df['season'] == s) & (pos_df['side'] == side) ]

        # Fill the df with the closest matching join_string found by the fuzzy matching
        df.loc[ (df.index.get_level_values(1) == s) & (df['side'] == side), 'join_string'] = tmp_snap_df['join_string'].apply(lambda x: GetClosestJoinString(tmp_pos_df, x))

df = df.reset_index()

# With common join strings now in hand, we can merge in the pfl position and player data from MFL
df = df.set_index(['join_string','season']).sort_index(level=[0,1],ascending=[True,True])
df = df.join( pos_df.set_index(['join_string','season']).sort_index(level=[0,1],ascending=[True,True])[['pos','player_mfl']] )

df = df.reset_index()

# Rename the position columns
df = df.rename(columns={'position':'pfr_position'})
df = df.rename(columns={'pos':'position'})

# Find the per-game snap ranks for each player, assign qualifying players weight of 1, and break ties at the boundary with fractional weight
df = df.set_index(['game_id','team']).sort_index(level=[0,1],ascending=[True,True])

df['offense_rank'] = df.groupby(['game_id','team']).apply(lambda x: x['offense_snaps'].rank(method='first',ascending=False)).values  #Snap rank
df['offense_thresh'] = df.groupby(['game_id','team']).apply(lambda x: x['offense_snaps'][ x['offense_rank'] == num_off ].values[0])  #Snap boundary
df['offense_weight'] = ( (df['offense_rank'] <= num_off).values | (df['offense_snaps'] == df['offense_thresh']).values ).astype(float)  #Initial weights
thresh_inds = df['offense_snaps'] == df['offense_thresh']
df.loc[ thresh_inds, 'offense_weight' ] = df[thresh_inds].groupby(['game_id','team']).apply(lambda x: np.sum(x['offense_rank'] <= num_off) / len(x) )  #Normalize weights by breaking ties

df['defense_rank'] = df.groupby(['game_id','team']).apply(lambda x: x['defense_snaps'].rank(method='first',ascending=False)).values  #Snap rank
df['defense_thresh'] = df.groupby(['game_id','team']).apply(lambda x: x['defense_snaps'][ x['defense_rank'] == num_def ].values[0])  #Snap boundary
df['defense_weight'] = ( (df['defense_rank'] <= num_def).values | (df['defense_snaps'] == df['defense_thresh']).values ).astype(float)  #Initial weights
thresh_inds = df['defense_snaps'] == df['defense_thresh']
df.loc[ thresh_inds, 'defense_weight' ] = df[thresh_inds].groupby(['game_id','team']).apply(lambda x: np.sum(x['defense_rank'] <= num_def) / len(x) )  #Normalize weights by breaking ties

df = df.reset_index()

# Sort by season, week, game_id, and team for cleaner display
df = df.set_index(['season','week','game_id','team','offense_snaps','defense_snaps']).sort_index(level=[0,1,2,3,4,5],ascending=[True,True,True,True,False,False]).reset_index()

# Save a copy of the raw data
df.to_csv(f'{save_label}_Raw.csv', index=False)

df

2024 OFF
2024 DEF
2023 OFF
2023 DEF
2022 OFF
2022 DEF


,season,week,game_id,team,offense_snaps,defense_snaps,join_string,pfr_player_id,player_pfr,pfr_position,side,position,player_mfl,offense_rank,offense_thresh,offense_weight,defense_rank,defense_thresh,defense_weight
0,2022,1,2022_01_BAL_NYJ,BAL,56.0,0.0,Lamar Jackson BAL QB,JackLa00,Lamar Jackson,QB,OFF,QB,Lamar Jackson,1.0,25.0,1.0,31.0,41.0,0.0
1,2022,1,2022_01_BAL_NYJ,BAL,47.0,0.0,Mark Andrews BAL TE,AndrMa00,Mark Andrews,TE,OFF,TE,Mark Andrews,2.0,25.0,1.0,32.0,41.0,0.0
2,2022,1,2022_01_BAL_NYJ,BAL,37.0,0.0,Rashod Bateman BAL WR,BateRa00,Rashod Bateman,WR,OFF,WR,Rashod Bateman,3.0,25.0,1.0,34.0,41.0,0.0
3,2022,1,2022_01_BAL_NYJ,BAL,33.0,0.0,Kenyan Drake BAL RB,DrakKe00,Kenyan Drake,RB,OFF,RB,Kenyan Drake,4.0,25.0,1.0,29.0,41.0,0.0
4,2022,1,2022_01_BAL_NYJ,BAL,29.0,0.0,Devin Duvernay BAL WR,DuveDe00,Devin Duvernay,WR,OFF,WR,Devin Duvernay,5.0,25.0,1.0,22.0,41.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54604,2024,17,2024_17_TEN_JAX,TEN,0.0,1.0,James Williams TEN LB,WillJa15,James Williams,LB,DEF,LB,James Williams,22.0,25.0,0.0,19.0,25.0,0.0
54605,2024,17,2024_17_TEN_JAX,TEN,0.0,0.0,Cedric Gray TEN LB,GrayCe00,Cedric Gray,LB,DEF,LB,Cedric Gray,15.0,25.0,0.0,22.0,25.0,0.0
54606,2024,17,2024_17_TEN_JAX,TEN,0.0,0.0,Justin Hardee TEN CB,HardJu01,Justin Hardee,CB,DEF,CB,Justin Hardee,27.0,25.0,0.0,27.0,25.0,0.0
54607,2024,17,2024_17_TEN_JAX,TEN,0.0,0.0,Kendell Brooks TEN S,BrooKe02,Kendell Brooks,S,DEF,S,Kendell Brooks,28.0,25.0,0.0,28.0,25.0,0.0


In [11]:
# Calculate the mean value for each season-team-position by summing over weights and dividing by number of games
df = df.set_index(['season','team','position']).sort_index(level=[0,1,2],ascending=[True,True,True])

result_off = df['offense_weight'].groupby(['season','team','position']).sum() / df.groupby(['season','team']).apply(lambda x: len(set(x['week'])))
result_off = result_off[ result_off >= 0.1 ]

result_def = df['defense_weight'].groupby(['season','team','position']).sum() / df.groupby(['season','team']).apply(lambda x: len(set(x['week'])))
result_def = result_def[ result_def >= 0.1 ]

df = df.reset_index()

# Pivot results out to columns to put into a nice tabular form
result_off = result_off.unstack(level=-1)
result_def = result_def.unstack(level=-1)

# Calculate values for DL and DB positions
result_def['DL'] = result_def['DT'] + result_def['DE']
result_def['DB'] = result_def['CB'] + result_def['S']

# Cleanup the table, listing the sorted values and season-teams for each position
result_dic = {}

for c in ['QB','RB','WR','TE']:
    tmp_df = result_off[c].reset_index()
    
    tmp_df[f'{c} Team'] = tmp_df['season'].map(str) + " " + tmp_df['team']
    tmp_df = tmp_df[[c, f'{c} Team']].rename(columns={c:f'{c} Value'})
    tmp_df = tmp_df.sort_values(f'{c} Value', ascending=False).reset_index(drop=True)

    result_dic[c] = tmp_df

for c in ['DT','DE','LB','CB','S','DL','DB']:
    tmp_df = result_def[c].reset_index()
    
    tmp_df[f'{c} Team'] = tmp_df['season'].map(str) + " " + tmp_df['team']
    tmp_df = tmp_df[[c, f'{c} Team']].rename(columns={c:f'{c} Value'})
    tmp_df = tmp_df.sort_values(f'{c} Value', ascending=False).reset_index(drop=True)

    result_dic[c] = tmp_df

# Save the table to a csv
result = pd.concat(result_dic, axis=1, ignore_index=False)
result.columns = result.columns.droplevel(0)

result.to_csv(f'{save_label}_Lineups.csv', index=False)

In [12]:
# Full results
result

,QB Value,QB Team,RB Value,RB Team,WR Value,WR Team,TE Value,TE Team,DT Value,DT Team,...,LB Value,LB Team,CB Value,CB Team,S Value,S Team,DL Value,DL Team,DB Value,DB Team
0,1.12500,2023 NOS,1.71875,2024 TBB,3.625000,2023 TBB,2.53125,2023 ATL,3.25000,2024 SEA,...,2.93750,2022 LAC,3.625000,2022 PHI,3.62500,2023 DAL,5.50000,2022 JAC,6.625000,2023 DAL
1,1.09375,2024 NYG,1.68750,2024 CHI,3.593750,2023 DAL,2.25000,2024 NOS,3.18750,2023 ATL,...,2.87500,2022 DAL,3.531250,2022 IND,3.09375,2024 HOU,5.40625,2023 CAR,5.875000,2022 DAL
2,1.06250,2022 ARI,1.68750,2022 CLE,3.562500,2023 BAL,2.21875,2024 ARI,3.15625,2023 CAR,...,2.81250,2022 ARI,3.500000,2022 CAR,3.06250,2024 KCC,5.37500,2022 MIA,5.687500,2022 NEP
3,1.06250,2023 WAS,1.62500,2023 PHI,3.531250,2022 CHI,2.18750,2024 MIA,3.12500,2023 LAR,...,2.78125,2024 DET,3.437500,2023 TBB,3.00000,2022 BAL,5.31250,2022 ATL,5.500000,2024 WAS
4,1.06250,2023 ATL,1.62500,2022 DAL,3.500000,2024 GBP,2.09375,2022 NOS,3.12500,2022 ARI,...,2.75000,2023 BAL,3.433333,2022 BUF,3.00000,2024 MIN,5.25000,2024 MIN,5.437500,2022 CAR
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,1.00000,2022 IND,1.00000,2023 JAC,2.625000,2024 PIT,1.18750,2022 CHI,1.71875,2023 CLE,...,1.71875,2023 CAR,2.312500,2022 SFO,1.62500,2022 IND,4.25000,2022 NEP,4.625000,2024 DET
92,1.00000,2023 PHI,0.96875,2024 DEN,2.520833,2022 ATL,1.15625,2023 MIA,1.68750,2023 DET,...,1.68750,2023 LAR,2.218750,2023 MIN,1.62500,2024 TBB,4.21875,2023 BAL,4.562500,2022 ATL
93,1.00000,2023 PIT,0.93750,2024 NOS,2.406250,2024 MIA,1.12500,2022 LAR,1.65625,2023 IND,...,1.59375,2023 MIN,2.187500,2023 DET,1.56250,2023 TBB,4.18750,2022 LAC,4.541667,2022 MIA
94,1.00000,2022 NYG,0.93750,2024 BUF,2.406250,2022 BAL,1.06250,2023 BAL,1.43750,2023 DAL,...,1.31250,2024 MIN,2.187500,2022 BAL,1.37500,2022 PHI,4.06250,2023 DAL,4.437500,2022 JAC


In [13]:
# Team 2
result.iloc[1]

QB Value     1.09375
QB Team     2024 NYG
RB Value      1.6875
RB Team     2024 CHI
WR Value     3.59375
WR Team     2023 DAL
TE Value        2.25
TE Team     2024 NOS
DT Value      3.1875
DT Team     2023 ATL
DE Value       3.125
DE Team     2024 MIN
LB Value       2.875
LB Team     2022 DAL
CB Value     3.53125
CB Team     2022 IND
S Value      3.09375
S Team      2024 HOU
DL Value     5.40625
DL Team     2023 CAR
DB Value       5.875
DB Team     2022 DAL
Name: 1, dtype: object

In [14]:
# Team 48 (half of median)
result.iloc[47]

QB Value         1.0
QB Team     2024 SFO
RB Value     1.21875
RB Team     2023 ARI
WR Value      3.0625
WR Team     2023 PIT
TE Value      1.6875
TE Team     2022 CLE
DT Value      2.4375
DT Team     2022 WAS
DE Value      2.3125
DE Team     2022 BAL
LB Value      2.0625
LB Team     2022 NEP
CB Value     2.90625
CB Team     2022 KCC
S Value        2.125
S Team      2022 MIA
DL Value       4.875
DL Team     2022 CHI
DB Value      5.0625
DB Team     2024 DEN
Name: 47, dtype: object

In [15]:
# Team 49 (half of median)
result.iloc[48]

QB Value         1.0
QB Team     2022 BUF
RB Value     1.21875
RB Team     2022 KCC
WR Value      3.0625
WR Team     2023 NYJ
TE Value    1.666667
TE Team     2022 CIN
DT Value    2.427083
DT Team     2024 NEP
DE Value      2.3125
DE Team     2022 NYJ
LB Value      2.0625
LB Team     2024 HOU
CB Value       2.875
CB Team     2024 SFO
S Value        2.125
S Team      2024 CIN
DL Value     4.84375
DL Team     2023 IND
DB Value      5.0625
DB Team     2023 LAC
Name: 48, dtype: object

In [16]:
# Team 95
result.iloc[94]

QB Value         1.0
QB Team     2022 NYG
RB Value      0.9375
RB Team     2024 BUF
WR Value     2.40625
WR Team     2022 BAL
TE Value      1.0625
TE Team     2023 BAL
DT Value      1.4375
DT Team     2023 DAL
DE Value     1.59375
DE Team     2022 DET
LB Value      1.3125
LB Team     2024 MIN
CB Value      2.1875
CB Team     2022 BAL
S Value        1.375
S Team      2022 PHI
DL Value      4.0625
DL Team     2023 DAL
DB Value      4.4375
DB Team     2022 JAC
Name: 94, dtype: object

In [20]:
# Examine instances where player names don't match
print(len(df[ (df['player_pfr'] != df['player_mfl']) & (df['offense_weight'] + df['defense_weight'] > 0) ]))

134


In [21]:
df[ (df['player_pfr'] != df['player_mfl']) & (df['offense_weight'] + df['defense_weight'] > 0) ].head(60)

,season,team,position,week,game_id,offense_snaps,defense_snaps,join_string,pfr_player_id,player_pfr,pfr_position,side,player_mfl,offense_rank,offense_thresh,offense_weight,defense_rank,defense_thresh,defense_weight
598,2022,ATL,CB,5,2022_05_ATL_TB,0.0,53.0,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,S,DEF,Dee Alford,21.0,21.0,0.0,8.0,38.0,1.0
2363,2022,CAR,DT,1,2022_01_CLE_CAR,0.0,48.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,26.0,18.0,0.0,10.0,34.0,1.0
2367,2022,CAR,DT,2,2022_02_CAR_NYG,0.0,54.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,27.0,18.0,0.0,8.0,38.0,1.0
2372,2022,CAR,DT,3,2022_03_NO_CAR,0.0,51.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,29.0,24.0,0.0,7.0,37.0,1.0
2377,2022,CAR,DT,4,2022_04_ARI_CAR,0.0,60.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,30.0,22.0,0.0,7.0,32.0,1.0
2382,2022,CAR,DT,5,2022_05_SF_CAR,0.0,49.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,29.0,27.0,0.0,7.0,27.0,1.0
2388,2022,CAR,DT,6,2022_06_CAR_LA,0.0,53.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,27.0,23.0,0.0,5.0,30.0,1.0
2398,2022,CAR,DT,8,2022_08_CAR_ATL,0.0,55.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,29.0,38.0,0.0,7.0,20.0,1.0
2401,2022,CAR,DT,9,2022_09_CAR_CIN,0.0,55.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,29.0,18.0,0.0,7.0,26.0,1.0
2407,2022,CAR,DT,10,2022_10_ATL_CAR,0.0,49.0,Matt Ioannidis CAR DT,IoanMa01,Matthew Ioannidis,DT,DEF,Matt Ioannidis,29.0,26.0,0.0,9.0,25.0,1.0


In [22]:
df[ (df['player_pfr'] != df['player_mfl']) & (df['offense_weight'] + df['defense_weight'] > 0) ].head(120).tail(60)

,season,team,position,week,game_id,offense_snaps,defense_snaps,join_string,pfr_player_id,player_pfr,pfr_position,side,player_mfl,offense_rank,offense_thresh,offense_weight,defense_rank,defense_thresh,defense_weight
27837,2023,LAR,CB,7,2023_07_PIT_LA,0.0,59.0,Decobie Durant LAR CB,DuraDe01,Cobie Durant,CB,DEF,Decobie Durant,17.0,29.0,0.0,2.0,29.0,1.0
27857,2023,LAR,CB,12,2023_12_LA_ARI,0.0,32.0,Decobie Durant LAR CB,DuraDe01,Cobie Durant,CB,DEF,Decobie Durant,20.0,24.0,0.0,11.0,31.0,1.0
27867,2023,LAR,CB,14,2023_14_LA_BAL,0.0,29.0,Decobie Durant LAR CB,DuraDe01,Cobie Durant,CB,DEF,Decobie Durant,19.0,19.0,0.0,12.0,29.0,1.0
27875,2023,LAR,CB,16,2023_16_NO_LA,0.0,50.0,Decobie Durant LAR CB,DuraDe01,Cobie Durant,CB,DEF,Decobie Durant,19.0,17.0,0.0,7.0,20.0,1.0
27879,2023,LAR,CB,17,2023_17_LA_NYG,0.0,53.0,Decobie Durant LAR CB,DuraDe01,Cobie Durant,CB,DEF,Decobie Durant,19.0,4.0,0.0,10.0,21.0,1.0
33505,2023,SEA,CB,4,2023_04_SEA_NYG,0.0,73.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,35.0,29.0,0.0,3.0,38.0,1.0
33510,2023,SEA,CB,6,2023_06_SEA_CIN,0.0,54.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,34.0,32.0,0.0,4.0,24.0,1.0
33515,2023,SEA,CB,7,2023_07_ARI_SEA,0.0,66.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,34.0,34.0,0.0,5.0,23.0,1.0
33520,2023,SEA,CB,8,2023_08_CLE_SEA,0.0,76.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,35.0,24.0,0.0,4.0,43.0,1.0
33525,2023,SEA,CB,9,2023_09_SEA_BAL,0.0,65.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,35.0,24.0,0.0,5.0,38.0,1.0


In [23]:
df[ (df['player_pfr'] != df['player_mfl']) & (df['offense_weight'] + df['defense_weight'] > 0) ].tail(60)

,season,team,position,week,game_id,offense_snaps,defense_snaps,join_string,pfr_player_id,player_pfr,pfr_position,side,player_mfl,offense_rank,offense_thresh,offense_weight,defense_rank,defense_thresh,defense_weight
33551,2023,SEA,CB,14,2023_14_SEA_SF,0.0,54.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,35.0,27.0,0.0,4.0,30.0,1.0
33561,2023,SEA,CB,16,2023_16_SEA_TEN,0.0,66.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,33.0,26.0,0.0,4.0,25.0,1.0
33568,2023,SEA,CB,17,2023_17_PIT_SEA,0.0,71.0,Tariq Woolen SEA CB,WoolTa00,Riq Woolen,CB,DEF,Tariq Woolen,34.0,22.0,0.0,5.0,26.0,1.0
36190,2023,WAS,S,15,2023_15_WAS_LA,0.0,75.0,Jartavius Martin WAS S,MartJa04,Quan Martin,CB,DEF,Jartavius Martin,25.0,24.0,0.0,3.0,35.0,1.0
36196,2023,WAS,S,16,2023_16_WAS_NYJ,0.0,62.0,Jartavius Martin WAS S,MartJa04,Quan Martin,CB,DEF,Jartavius Martin,26.0,27.0,0.0,9.0,39.0,1.0
36199,2023,WAS,S,17,2023_17_SF_WAS,0.0,63.0,Jartavius Martin WAS S,MartJa04,Quan Martin,CB,DEF,Jartavius Martin,24.0,23.0,0.0,5.0,34.0,1.0
36922,2024,ATL,CB,1,2024_01_PIT_ATL,0.0,31.0,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,Dee Alford,18.0,23.0,0.0,11.0,31.0,0.5
36926,2024,ATL,CB,2,2024_02_ATL_PHI,0.0,53.0,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,Dee Alford,17.0,16.0,0.0,6.0,36.0,1.0
36933,2024,ATL,CB,3,2024_03_KC_ATL,0.0,48.0,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,Dee Alford,19.0,19.0,0.0,7.0,35.0,1.0
36939,2024,ATL,CB,4,2024_04_NO_ATL,0.0,38.0,Dee Alford ATL CB,AlfoDe00,DeAundre Alford,CB,DEF,Dee Alford,18.0,21.0,0.0,10.0,37.0,1.0
